<a href="https://colab.research.google.com/github/Manish927/EDA-Data-Science/blob/feat/census-income/census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# !pip install ucimlrepo
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
from seaborn import countplot
import seaborn as sns

class DatasetWrangler:
    def __init__(self, uci_id, missing_flag='?'):
        """
        Initializes the wrangler by fetching data and flagging missing values.
        """
        print(f"Fetching dataset {uci_id}...")
        dataset = fetch_ucirepo(id=uci_id)

        # Combine X and y into one dataframe
        self.df = pd.concat([dataset.data.features, dataset.data.targets], axis=1)
        self.target_name = dataset.data.targets.columns[0]

        # Immediately set the missing flag to NaN for analysis
        self.df.replace(missing_flag, np.nan, inplace=True)
        print("Data loaded. Missing flags handled.")

    def display_info(self):
        """Displays the structural info of the dataframe (types, memory, etc.)"""
        print("\n--- 📋 Dataframe Information ---")
        self.df.info()
        return self

    def show_missing_stats(self):
        """Calculates and prints the percentage of missing values per column."""
        stats = (self.df.isnull().mean() * 100).round(2)
        stats = stats[stats > 0].sort_values(ascending=False)

        if stats.empty:
            print("No missing values detected!")
        else:
            print("\n--- Missing Value Percentages ---")
            print(stats)
        return self

    def plot_missing_scatterness(self):
        """Generates the Matrix and Heatmap to visualize missing data distribution."""
        print("\nGenerating missingness visualizations...")

        # Matrix Plot (The 'Scatteredness' barcode)
        plt.figure(figsize=(10, 4))
        msno.matrix(self.df)
        plt.title(f"Missing Values Matrix (Dataset ID)", fontsize=16)
        plt.show()

        # Heatmap (The Correlation of missingness)
        if self.df.isnull().sum().sum() > 0:
            plt.figure(figsize=(8, 6))
            msno.heatmap(self.df)
            plt.title("Missingness Correlation Heatmap", fontsize=16)
            plt.show()
        else:
            print("Skipping heatmap: No missing values to correlate.")
        return self

    def get_data(self):
        """Returns the current state of the dataframe."""
        return self.df

    def plot_distributions(self):
        """
        Automatically creates histograms for numerical data
        and count plots for categorical data.
        """
        print("\n📊 Generating distributions for all columns...")

        # Identify column types
        num_cols = self.df.select_dtypes(include=['number']).columns
        cat_cols = self.df.select_dtypes(include=['object', 'category']).columns

        # Plot Numerical Columns (Histograms)
        if len(num_cols) > 0:
            self.df[num_cols].hist(bins=20, figsize=(15, 10), layout=(-1, 3),
                                   edgecolor='black', color='skyblue')
            plt.suptitle("Numerical Feature Distributions (Histograms)", fontsize=20)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.show()

        # Plot Categorical Columns (Bar Charts)
        if len(cat_cols) > 0:
            # We limit to top 10 categories if a column has too many (like 'native-country')
            for col in cat_cols:
                plt.figure(figsize=(10, 4))
                sns.countplot(data=self.df, y=col, order=self.df[col].value_counts().iloc[:10].index, palette='viridis')
                plt.title(f"Top 10 Categories in {col}")
                plt.show()

        return self

In [ ]:
# For Census Income (ID 2), missing values are '?'
census_data = DatasetWrangler(uci_id=2, missing_flag='?')
# Chain the functions to see the stats and plots
census_data.display_info().show_missing_stats().plot_distributions()


In [ ]:
#display workclass data
census_data.df['native-country'].value_counts()

# Since we need to fill the values in missing columns, we can use iterative imputer, knn(efficient), random forest to predict the missing values.